#資料讀取

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/My Drive/#PTT_Text Mining/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd

In [ ]:
input_file = open(data_path + 'Data/' + 'df.json', 'r') #資料位置
data = json.load(input_file)
df = json_normalize(data)
df.head(2)

In [ ]:
df['Board'].value_counts()

Sad          25820
happy        24744
Gossiping    21572
prozac       20440
Name: Board, dtype: int64

#刪除遺漏值

In [ ]:
#看板活動、網頁格式不符無法抓取
df_na = df[df.isna().any(axis=1)]
df_na['Board'].value_counts()

prozac       198
Sad          147
Gossiping    104
happy         58
Name: Board, dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df['Board'].value_counts()

Gossiping    47212
Sad          27492
happy        25082
prozac       22181
Name: Board, dtype: int64

In [ ]:
df.shape

(121967, 6)

#文章類別

In [ ]:
df['Tag'] = df['Title'].str.extract('\[(.*)\]')  
df.head()

#刪除無關文章

In [ ]:
#刪除回覆、轉發的文章

In [ ]:
df_filter = df[df['Title'].str.startswith("Re:", "Fw:")]
df_filter['Board'].value_counts()

Gossiping    15759
prozac        1078
Sad            191
happy           59
Name: Board, dtype: int64

In [ ]:
myfilter = df['Title'].str.startswith("Re:", "Fw:")
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Gossiping    31548
Sad          27312
happy        25025
prozac       21186
Name: Board, dtype: int64

In [ ]:
#刪除看版公告的文章

In [ ]:
df_filter = df[df['Title'].str.contains('公告')]
df_filter['Board'].value_counts()

Gossiping    846
Sad           36
prozac        27
happy          7
Name: Board, dtype: int64

In [ ]:
myfilter = df['Title'].str.contains('公告')
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Gossiping    30702
Sad          27276
happy        25018
prozac       21159
Name: Board, dtype: int64

In [ ]:
#刪除prozac版的無關文章

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='自介')
df[myfilter].head()

In [ ]:
df[myfilter].shape

(26, 6)

In [ ]:
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Gossiping    30702
Sad          27276
happy        25018
prozac       21133
Name: Board, dtype: int64

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='醫療')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='經驗')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='資訊')
df[myfilter].head()

In [ ]:
df[myfilter].shape

(137, 6)

In [ ]:
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Gossiping    30702
Sad          27276
happy        25018
prozac       20996
Name: Board, dtype: int64

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='問題')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='轉錄')
df[myfilter].head()

In [ ]:
df[myfilter].shape

(6, 6)

In [ ]:
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Gossiping    30702
Sad          27276
happy        25018
prozac       20990
Name: Board, dtype: int64

In [ ]:
myfilter = (df['Board']=='prozac') & (df['Tag']=='閒聊')
df[myfilter].head()

In [ ]:
#刪除Gossiping版的無關文章

In [ ]:
myfilter = (df['Board']=='Gossiping') & (df['Tag']=='爆卦')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='Gossiping') & (df['Tag']=='問卦')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='Gossiping') & (df['Tag']=='新聞')
df[myfilter].head()

In [ ]:
df[myfilter].shape

(7634, 6)

In [ ]:
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Sad          27276
happy        25018
Gossiping    23068
prozac       20990
Name: Board, dtype: int64

In [ ]:
#刪除Sad版的無關文章

In [ ]:
myfilter = (df['Board']=='Sad') & (df['Tag']=='心情')
df[myfilter].head()

In [ ]:
myfilter = (df['Board']=='Sad') & (df['Tag']=='分享')
df[myfilter].head()

In [ ]:
df[myfilter].shape

(100, 6)

In [ ]:
df = df[~myfilter]

In [ ]:
df['Board'].value_counts()

Sad          27176
happy        25018
Gossiping    23068
prozac       20990
Name: Board, dtype: int64

In [ ]:
myfilter = (df['Board']=='Sad') & (df['Tag']=='討論')
df[myfilter].head()

#「憂鬱傾向」欄位：有→1；無→0

In [ ]:
df['Depressive'] = df['Board'].apply(lambda x: 1 if x=='prozac' else 0)  
df.head()

#「情緒」欄位：正向→1；負向→0

In [ ]:
df.loc[(df['Tag']=='晴天') & (df['Board']=='prozac'), 'Emotion'] = 1
df.loc[(df['Tag']=='陰天') & (df['Board']=='prozac'), 'Emotion'] = 0
df.head()

In [ ]:
df.loc[(df['Board']=='happy'), 'Emotion'] = 1
df.loc[(df['Board']=='Sad'), 'Emotion'] = 0
df.head()

#文本符號處理

In [ ]:
import re

def preprocessor(text):

    #清除特定內容
    text = re.sub('(\n)|[a-z]|[A-Z]|[0-9]', ' ', text)

		#'[\W]+':清除標點符號
    text = re.sub('[\W]+', ' ', text)
    
    return text

In [ ]:
df['Format_Content'] = df['Content'].apply(preprocessor)
df.head()

#斷詞

In [ ]:
!pip install -U ckiptagger[tf, gdown]

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

ws = WS(data_path + "CkipTagger_data")

In [ ]:
#1 hr 40 min
df['Format_Content'] = ws(df['Format_Content'])

#刪除字數太少的文章：<30字

In [ ]:
myfilter = df['Content'].str.len() < 30
df[myfilter]['Board'].value_counts()

Gossiping    1064
prozac        550
Sad           240
happy         143
Name: Board, dtype: int64

In [ ]:
df = df[~myfilter]
df['Board'].value_counts()

Sad          26936
happy        24875
Gossiping    22004
prozac       20440
Name: Board, dtype: int64

#曾在憂鬱版發文的帳號，視為自覺有憂鬱傾向，因此排除在其他看版所發的文章。

In [ ]:
mylist = df[df['Board']=='prozac']['Author'].tolist()
myfilter = (df['Board']!='prozac') & (df['Author'].isin(mylist))
df[myfilter]['Board'].value_counts()

Sad          1116
Gossiping     432
happy         131
Name: Board, dtype: int64

In [ ]:
df = df[~myfilter]
df['Board'].value_counts()

Sad          25820
happy        24744
Gossiping    21572
prozac       20440
Name: Board, dtype: int64

#刪除超出時間區間的文章：2010/1/1 - 2019/12/31

In [ ]:
import datetime

In [ ]:
def GetTime(text):
  
  try:
    text = datetime.datetime.strptime(text, '%a %b %d %H:%M:%S %Y')
  except:
     text = None

  return text

In [ ]:
df['Format_Time'] = df['Time'].apply(GetTime)

In [ ]:
#確認缺漏值

In [ ]:
df[pd.isna(df['Format_Time'])==True].head()

In [ ]:
df[pd.isna(df['Format_Time'])==True]['Board'].value_counts()

prozac    3
Sad       1
Name: Board, dtype: int64

In [ ]:
#刪除缺漏值

In [ ]:
df.dropna(subset=['Format_Time'], inplace=True)

In [ ]:
df['Board'].value_counts()

Sad          25819
happy        24744
Gossiping    21572
prozac       20437
Name: Board, dtype: int64

In [ ]:
#刪除超出時間區間的文章：2010/1/1 - 2019/12/31
#df['Format_Time'] = pd.to_datetime(df['Format_Time'], unit='ms')
start_date = '2010-01-01 00:00:00'
end_date = '2019-12-31 11:59:59'
myfilter = (df['Format_Time'] >= start_date) & (df['Format_Time'] <= end_date)
df[~myfilter].head()

In [ ]:
df[~myfilter]['Board'].value_counts()

prozac       157
Gossiping     69
happy         45
Sad           28
Name: Board, dtype: int64

In [ ]:
df = df[myfilter]
df['Board'].value_counts()

Sad          25791
happy        24699
Gossiping    21503
prozac       20280
Name: Board, dtype: int64

#刪除list中的空白

In [ ]:
def ClearListBlank(mylist):
  mylist = [x for x in mylist if x != ' ']
  return mylist

In [ ]:
df['Format_Content'].apply(ClearListBlank)
df.head()

In [ ]:
#設定'Format_Content'欄位以利分析

In [ ]:
df['Format_Content'] = df['Format_Content'].apply(lambda x: " ".join(x) )
df.head()

#標註詞性

In [ ]:
!pip install -U ckiptagger[tf,gdown]

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from ckiptagger import data_utils, construct_dictionary, POS

pos = POS(data_path + "CkipTagger_data")

In [ ]:
# 2 hr 8 min
df['Format_POS_Content'] = pos(df['Format_Content'])
df.head()